<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [6]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [7]:
import folium
import pandas as pd

In [8]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [9]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [10]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i, row in launch_sites_df.iterrows():
    ls_name = row['Launch Site']
    ls_lat  = row['Lat']
    ls_long = row['Long']
    
    coord = [ls_lat, ls_long]
    circ = folium.Circle(coord, radius=1000, color='#000000', fill=True).add_child(folium.Popup(ls_name + ' region'))
    mark = folium.map.Marker(coord, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ls_name, ))
    
    site_map.add_child(circ)
    site_map.add_child(mark)
    
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [17]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def MarkColor(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(MarkColor)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for i, record in spacex_df.iterrows():
    ls_name = record['Launch Site']
    ls_lat  = record['Lat']
    ls_long = record['Long']
    coord = [ls_lat, ls_long]
    
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.map.Marker(coord, icon=folium.Icon(color='white', icon_color=record['marker_color'],icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ls_name, ))
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
#distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
dist=calculate_distance(28.562074, -80.577256, 28.562749, -80.56794)
dist

0.913197870865835

In [22]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
coord = [28.562074, -80.577256]
dist_marker = folium.Marker(
    coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist),
    )
)
site_map.add_child(dist_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [23]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coord=[[28.562074, -80.577256],[28.562749, -80.56781]]
lines=folium.PolyLine(locations=coord, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [24]:
LaunchSitesDict = launch_sites_df.set_index('Launch Site').T.to_dict('list')
LaunchSitesDict

{'CCAFS LC-40': [28.56230197, -80.57735648],
 'CCAFS SLC-40': [28.56319718, -80.57682003],
 'KSC LC-39A': [28.57325457, -80.64689529],
 'VAFB SLC-4E': [34.63283416, -120.6107455]}

In [25]:
#CCAFS LC-40
CC_LC_LS=[28.56230197, -80.57735648]
CC_LC_Rail=[28.57207, -80.58528]
CC_LC_High=[28.56230197, -80.60922]
CC_LC_Coast=[28.56258, -80.56765]
CC_LC_City=[28.66288, -81.35925]

dCC_LC_Rail=calculate_distance(CC_LC_LS[0],CC_LC_LS[1], CC_LC_Rail[0],CC_LC_Rail[1])
dCC_LC_High=calculate_distance(CC_LC_LS[0],CC_LC_LS[1], CC_LC_High[0],CC_LC_High[1])
dCC_LC_Coast=calculate_distance(CC_LC_LS[0],CC_LC_LS[1], CC_LC_Coast[0],CC_LC_Coast[1])
dCC_LC_City=calculate_distance(CC_LC_LS[0],CC_LC_LS[1], CC_LC_City[0],CC_LC_City[1])

#CCAFS SLC-40
CC_SLC_LS=[28.56319718, -80.57682003]
CC_SLC_Rail=[28.57367, -80.58472]
CC_SLC_High=[28.563197, -80.64651]
CC_SLC_Coast=[28.563197, -80.56772]
CC_SLC_City=[28.66288, -81.35925]

dCC_SLC_Rail=calculate_distance(CC_SLC_LS[0],CC_SLC_LS[1], CC_SLC_Rail[0],CC_SLC_Rail[1])
dCC_SLC_High=calculate_distance(CC_SLC_LS[0],CC_SLC_LS[1], CC_SLC_High[0],CC_SLC_High[1])
dCC_SLC_Coast=calculate_distance(CC_SLC_LS[0],CC_SLC_LS[1], CC_SLC_Coast[0],CC_SLC_Coast[1])
dCC_SLC_City=calculate_distance(CC_SLC_LS[0],CC_SLC_LS[1], CC_SLC_City[0],CC_SLC_City[1])

#KSC
KSC_LS=[28.57325457, -80.64689529]
KSC_Rail=[28.573255, -80.65411]
KSC_High=[28.573255, -80.646895]
KSC_Coast=[28.573255, -80.60669]
KSC_City=[28.53748, -81.38672]

dKSC_Rail=calculate_distance(KSC_LS[0],KSC_LS[1], KSC_Rail[0],KSC_Rail[1])
dKSC_High=calculate_distance(KSC_LS[0],KSC_LS[1], KSC_High[0],KSC_High[1])
dKSC_Coast=calculate_distance(KSC_LS[0],KSC_LS[1], KSC_Coast[0],KSC_Coast[1])
dKSC_City=calculate_distance(KSC_LS[0],KSC_LS[1], KSC_City[0],KSC_City[1])

#VAFB SLC-4E
VAFB_LS=[34.63283416, -120.6107455]
VAFB_Rail=[34.63141, -120.62568]
VAFB_High=[34.66992, -120.45753]
VAFB_Coast=[34.6336, -120.62606]
VAFB_City=[34.63658, -120.4542]

dVAFB_Rail=calculate_distance(VAFB_LS[0],VAFB_LS[1], VAFB_Rail[0],VAFB_Rail[1])
dVAFB_High=calculate_distance(VAFB_LS[0],VAFB_LS[1], VAFB_High[0],VAFB_High[1])
dVAFB_Coast=calculate_distance(VAFB_LS[0],VAFB_LS[1], VAFB_Coast[0],VAFB_Coast[1])
dVAFB_City=calculate_distance(VAFB_LS[0],VAFB_LS[1], VAFB_City[0],VAFB_City[1])

# list of Launch_Site, dist_Rail, dist_High, dist_Coast, dist_City
site = ["CCAFS LC-40", "CCAFS SLC-40", "KSC LC-39A", "VAFB SLC-4E"]
dRail = [dCC_LC_Rail,dCC_SLC_Rail,dKSC_Rail,dVAFB_Rail]
dHigh = [dCC_LC_High,dCC_SLC_High,dKSC_High,dVAFB_High]
dCoast = [dCC_LC_Coast,dCC_SLC_Coast,dKSC_Coast,dVAFB_Coast]
dCity = [dCC_LC_City,dCC_SLC_City,dKSC_City,dVAFB_City]

# dictionary of lists 
dict = {'Launch Site': site, 'distance to Railway': dRail, 'distance to Highway': dHigh, 'distance to Coastline': dCoast, 'distance to City': dCity} 
df = pd.DataFrame(dict)
df

,Launch Site,distance to Railway,distance to Highway,distance to Coastline,distance to City
0,CCAFS LC-40,1.334019,3.112840,0.948757,77.164049
1,CCAFS SLC-40,1.397330,6.808158,0.889001,77.201193
2,KSC LC-39A,0.704753,0.000056,3.927363,72.389724
3,VAFB SLC-4E,1.375983,14.613598,1.404180,14.332950


In [26]:
#KSC_Rail_Dist
dmKSC_Rail = folium.Marker(
    KSC_Rail,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:blue;"><b>%s</b></div>' % "{:10.2f} KM|Railway".format(dKSC_Rail),
    )
)
site_map.add_child(dmKSC_Rail)

In [27]:
#KSC_Rail_Line
coordKSC_Rail=[KSC_LS,KSC_Rail]
lineKSC_Rail=folium.PolyLine(locations=coordKSC_Rail, weight=1,color="blue")
site_map.add_child(lineKSC_Rail)

In [28]:
#KSC_High_Dist
dmKSC_High = folium.Marker(
    KSC_High,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:red;"><b>%s</b></div>' % "{:10.2f} KM|Highway".format(dKSC_High),
    )
)
site_map.add_child(dmKSC_High)

In [29]:
#KSC_High_Line
coordKSC_High=[KSC_LS,KSC_High]
lineKSC_High=folium.PolyLine(locations=coordKSC_High, weight=1,color="red")
site_map.add_child(lineKSC_High)

In [30]:
#KSC_Coast_Dist
dmKSC_Coast = folium.Marker(
    KSC_Coast,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:orange;"><b>%s</b></div>' % "{:10.2f} KM|Coastline".format(dKSC_Coast),
    )
)
site_map.add_child(dmKSC_Coast)

In [31]:
#KSC_Coast_Line
coordKSC_Coast=[KSC_LS,KSC_Coast]
lineKSC_Coast=folium.PolyLine(locations=coordKSC_Coast, weight=1,color="orange")
site_map.add_child(lineKSC_Coast)

In [32]:
#KSC_City_Dist
dmKSC_City = folium.Marker(
    KSC_City,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:green;"><b>%s</b></div>' % "{:10.2f} KM|City".format(dKSC_City),
    )
)
site_map.add_child(dmKSC_City)

In [33]:
#KSC_City_Line
coordKSC_City=[KSC_LS,KSC_City]
lineKSC_City=folium.PolyLine(locations=coordKSC_City, weight=1,color="green")
site_map.add_child(lineKSC_City)

In [34]:
#CC_LC_Rail_Dist
dmCC_LC_Rail = folium.Marker(
    CC_LC_Rail,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:blue;"><b>%s</b></div>' % "{:10.2f} KM|Railway".format(dCC_LC_Rail),
    )
)
site_map.add_child(dmCC_LC_Rail)

In [35]:
#CC_LC_Rail_Line
coordCC_LC_Rail=[CC_LC_LS,CC_LC_Rail]
lineCC_LC_Rail=folium.PolyLine(locations=coordCC_LC_Rail, weight=1,color="blue")
site_map.add_child(lineCC_LC_Rail)

In [36]:
#CC_LC_High_Dist
dmCC_LC_High = folium.Marker(
    CC_LC_High,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:red;"><b>%s</b></div>' % "{:10.2f} KM|Highway".format(dCC_LC_High),
    )
)
site_map.add_child(dmCC_LC_High)

In [37]:
#CC_LC_High_Line
coordCC_LC_High=[CC_LC_LS,CC_LC_High]
lineCC_LC_High=folium.PolyLine(locations=coordCC_LC_High, weight=1,color="red")
site_map.add_child(lineCC_LC_High)

In [38]:
#CC_LC_Coast_Dist
dmCC_LC_Coast = folium.Marker(
    CC_LC_Coast,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:orange;"><b>%s</b></div>' % "{:10.2f} KM|Coast".format(dCC_LC_Coast),
    )
)
site_map.add_child(dmCC_LC_Coast)

In [39]:
#CC_LC_Coast_Line
coordCC_LC_Coast=[CC_LC_LS,CC_LC_Coast]
lineCC_LC_Coast=folium.PolyLine(locations=coordCC_LC_Coast, weight=1,color="orange")
site_map.add_child(lineCC_LC_Coast)

In [40]:
#CC_LC_City_Dist
dmCC_LC_City = folium.Marker(
    CC_LC_City,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:green;"><b>%s</b></div>' % "{:10.2f} KM|City".format(dCC_LC_City),
    )
)
site_map.add_child(dmCC_LC_City)

In [41]:
#CC_LC_City_Line
coordCC_LC_City=[CC_LC_LS,CC_LC_City]
lineCC_LC_City=folium.PolyLine(locations=coordCC_LC_City, weight=1,color="green")
site_map.add_child(lineCC_LC_City)

In [42]:
#CC_SLC_Rail_Dist
dmCC_SLC_Rail = folium.Marker(
    CC_SLC_Rail,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:blue;"><b>%s</b></div>' % "{:10.2f} KM|Railway".format(dCC_SLC_Rail),
    )
)
site_map.add_child(dmCC_SLC_Rail)

In [44]:
#CC_SLC_Rail_Line
coordCC_SLC_Rail=[CC_SLC_LS,CC_SLC_Rail]
lineCC_SLC_Rail=folium.PolyLine(locations=coordCC_SLC_Rail, weight=1,color="blue")
site_map.add_child(lineCC_SLC_Rail)

In [45]:
#CC_SLC_High_Dist
dmCC_SLC_High = folium.Marker(
    CC_SLC_High,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:red;"><b>%s</b></div>' % "{:10.2f} KM|Highway".format(dCC_SLC_High),
    )
)
site_map.add_child(dmCC_SLC_High)

In [46]:
#CC_SLC_High_Line
coordCC_SLC_High=[CC_SLC_LS,CC_SLC_High]
lineCC_SLC_High=folium.PolyLine(locations=coordCC_SLC_High, weight=1,color="red")
site_map.add_child(lineCC_SLC_High)

In [47]:
#CC_SLC_Coast_Dist
dmCC_SLC_Coast = folium.Marker(
    CC_SLC_Coast,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:orange;"><b>%s</b></div>' % "{:10.2f} KM|Coastway".format(dCC_SLC_Coast),
    )
)
site_map.add_child(dmCC_SLC_Coast)

In [48]:
#CC_SLC_Coast_Line
coordCC_SLC_Coast=[CC_SLC_LS,CC_SLC_Coast]
lineCC_SLC_Coast=folium.PolyLine(locations=coordCC_SLC_Coast, weight=1,color="orange")
site_map.add_child(lineCC_SLC_Coast)

In [49]:
#CC_SLC_City_Dist
dmCC_SLC_City = folium.Marker(
    CC_SLC_City,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:green;"><b>%s</b></div>' % "{:10.2f} KM|City".format(dCC_SLC_City),
    )
)
site_map.add_child(dmCC_SLC_City)

In [50]:
#CC_SLC_City_Line
coordCC_SLC_City=[CC_SLC_LS,CC_SLC_City]
lineCC_SLC_City=folium.PolyLine(locations=coordCC_SLC_City, weight=1,color="green")
site_map.add_child(lineCC_SLC_City)

In [51]:
#VAFB_Rail_Dist
dmVAFB_Rail = folium.Marker(
    VAFB_Rail,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:blue;"><b>%s</b></div>' % "{:10.2f} KM|Railway".format(dVAFB_Rail),
    )
)
site_map.add_child(dmVAFB_Rail)

In [52]:
#VAFB_Rail_Line
coordVAFB_Rail=[VAFB_LS,VAFB_Rail]
lineVAFB_Rail=folium.PolyLine(locations=coordVAFB_Rail, weight=1,color="blue")
site_map.add_child(lineVAFB_Rail)

In [53]:
#VAFB_High_Dist
dmVAFB_High = folium.Marker(
    VAFB_High,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:red;"><b>%s</b></div>' % "{:10.2f} KM|Highway".format(dVAFB_High),
    )
)
site_map.add_child(dmVAFB_High)

In [54]:
#VAFB_High_Line
coordVAFB_High=[VAFB_LS,VAFB_High]
lineVAFB_High=folium.PolyLine(locations=coordVAFB_High, weight=1,color="red")
site_map.add_child(lineVAFB_High)

In [55]:
#VAFB_Coast_Dist
dmVAFB_Coast = folium.Marker(
    VAFB_Coast,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:orange;"><b>%s</b></div>' % "{:10.2f} KM|Coast".format(dVAFB_Coast),
    )
)
site_map.add_child(dmVAFB_Coast)

In [56]:
#VAFB_Coast_Line
coordVAFB_Coast=[VAFB_LS,VAFB_Coast]
lineVAFB_Coast=folium.PolyLine(locations=coordVAFB_Coast, weight=1,color="orange")
site_map.add_child(lineVAFB_Coast)

In [57]:
#VAFB_City_Dist
dmVAFB_City = folium.Marker(
    VAFB_City,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12;color:green;"><b>%s</b></div>' % "{:10.2f} KM|City".format(dVAFB_City),
    )
)
site_map.add_child(dmVAFB_City)

In [58]:
#VAFB_City_Line
coordVAFB_City=[VAFB_LS,VAFB_City]
lineVAFB_City=folium.PolyLine(locations=coordVAFB_City, weight=1,color="green")
site_map.add_child(lineVAFB_City)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
